In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
from pathlib import Path
import xarray as xr
import tensorflow as tf
import datetime
from shared.models import *
from shared.utilities import *
from shared.training import train_and_evaluate, split_data_on_participants
import random
%env TF_FORCE_GPU_ALLOW_GROWTH=true
%env TF_GPU_ALLOCATOR=cuda_malloc_async

2023-08-31 13:07:06.003286: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-31 13:07:06.762115: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


env: TF_FORCE_GPU_ALLOW_GROWTH=true
env: TF_GPU_ALLOCATOR=cuda_malloc_async


### Set up data

In [7]:
data_path = Path("data/sat1/split_stage_data.nc")

data = xr.load_dataset(data_path)

In [8]:
data = data.where(data.cue == "SP")

In [9]:
train_data, val_data, test_data = split_data_on_participants(data, 60)

In [10]:
model = SAT1Base(len(data.channels), len(data.samples), len(data.labels))
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00005),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"],
)

In [11]:
train_and_evaluate(model, train_data, val_data, test_data, epochs=20)

Epoch 1/20
249/249 [==============================] - 9s 31ms/step - loss: 1.1437 - accuracy: 0.3434 - val_loss: 1.0970 - val_accuracy: 0.6204
Epoch 2/20
249/249 [==============================] - 8s 29ms/step - loss: 1.1104 - accuracy: 0.3650 - val_loss: 1.0227 - val_accuracy: 0.7119
Epoch 3/20
249/249 [==============================] - 8s 29ms/step - loss: 1.0043 - accuracy: 0.5030 - val_loss: 0.8114 - val_accuracy: 0.7309
Epoch 4/20
249/249 [==============================] - 8s 29ms/step - loss: 0.8460 - accuracy: 0.6250 - val_loss: 0.6572 - val_accuracy: 0.7363
Epoch 5/20
249/249 [==============================] - 8s 30ms/step - loss: 0.7687 - accuracy: 0.6556 - val_loss: 0.6059 - val_accuracy: 0.7439
Epoch 6/20
249/249 [==============================] - 8s 29ms/step - loss: 0.7290 - accuracy: 0.6752 - val_loss: 0.5589 - val_accuracy: 0.7584
Epoch 7/20
249/249 [==============================] - 8s 29ms/step - loss: 0.6795 - accuracy: 0.7116 - val_loss: 0.5160 - val_accuracy: 0.8133

In [42]:
# Run before re-training to clear up VRAM
import gc

gc.collect()
tf.keras.backend.clear_session()
del model